# Markups

In [1]:
# default_exp markup

In [2]:
# export
def isnamedtupleinstance(x):
    _type = type(x)
    bases = _type.__bases__
    if len(bases) != 1 or bases[0] != tuple:
        return False
    fields = getattr(_type, '_fields', None)
    if not isinstance(fields, tuple):
        return False
    return all(type(i)==str for i in fields)

def unpack(obj):
    if isinstance(obj, dict):
        return {key: unpack(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [unpack(value) for value in obj]
    elif isnamedtupleinstance(obj):
        return {key: unpack(value) for key, value in obj._asdict().items()}
    elif isinstance(obj, tuple):
        return tuple(unpack(value) for value in obj)
    else:
        return obj

In [3]:
# export
import json
from torch_snippets.paths import *
from torch_snippets.logger import *

def read_json(fpath):
    import json
    with open(fpath, 'r') as f:
        return json.load(f)

def write_json(obj, fpath):
    def set_default(obj):
        if isinstance(obj, set):
            return list(obj)
        raise TypeError

    logger.opt(depth=1).log("DEBUG", f"Dumping json to {fpath}")
    with open(fpath, "w") as f:
        json.dump(obj, f, indent=4, default=set_default)
    return P(fpath)

class AttrDict(object):
    def __init__(self, data):
        for name, value in data.items():
            setattr(self, name, self._wrap(value))

    def _wrap(self, value):
        if isinstance(value, (tuple, list, set, frozenset)):
            return type(value)([self._wrap(v) for v in value])
        else:
            return AttrDict(value) if isinstance(value, dict) else value

    __getitem__ = lambda self, x: getattr(self, x)
    __setitem__ = lambda self, k, v: setattr(self, k, self._wrap(v))

    def __repr__(self):
        return '{%s}' % str(', '.join("'%s': %s" % (k, repr(v)) for (k, v) in self.__dict__.items()))

    def __dir__(self):
        return self.__dict__.keys()

    def to_dict(self):
        d = {}
        for k in dir(self):
            v = self[k]
            if isinstance(v, AttrDict):
                v = v.to_dict()
            d[k] = v
        return d

    def pretty(self, *args, **kwargs):
        pretty_json(self.to_dict(), *args, **kwargs)

    def __eq__(self, other):
        return AttrDict(other).to_dict() == self.to_dict()


def pretty_json(i, fpath=None, indent=4):
    def set_default(obj):
        if isinstance(obj, set):
            return list(obj)
    assert isinstance(i, (dict, list))
    i = unpack(i)
    dump = json.dumps(i, indent=indent, default=set_default)
    if fpath:
        makedir(parent(fpath))
        logger.info(f'Dumped a pretty file to {fpath}')
        with open(fpath, 'w') as f:
            json.dump(i, f, indent=indent, default=set_default)
            return
    return logger.opt(depth=1).log('DEBUG', f'\n{dump}')

2021-09-30 17:41:13.514 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'


In [4]:
x = {'abc': {'b': 10, 'c': 11}, 'd': {'e': {'f': [2, {'abc': 'pqrs'}, 2.234]}}}

y = AttrDict(x)

assert y.abc.b == 10
assert y.d.e.f == [2,{'abc': 'pqrs'},2.234]

y.d.e.g = 11

assert y.to_dict() == {'abc': {'b': 10, 'c': 11}, 'd': {'e': {'f': [2, {'abc': 'pqrs'}, 2.234], 'g': 11}}}

y.pretty(indent=2)

2021-09-30 17:41:13.523 | DEBUG    | __main__:pretty:52 - 
{
  "abc": {
    "b": 10,
    "c": 11
  },
  "d": {
    "e": {
      "f": [
        2,
        null,
        2.234
      ],
      "g": 11
    }
  }
}


In [5]:
d = [1,{1:1,2:2},3]

pretty_json({1:1,2:2})
pretty_json(d)

f = write_json(d, '/tmp/test.json')
print(f)
read_json(f)

2021-09-30 17:41:13.530 | DEBUG    | __main__:<module>:3 - 
{
    "1": 1,
    "2": 2
}
2021-09-30 17:41:13.532 | DEBUG    | __main__:<module>:4 - 
[
    1,
    {
        "1": 1,
        "2": 2
    },
    3
]
2021-09-30 17:41:13.533 | DEBUG    | __main__:<module>:6 - Dumping json to /tmp/test.json


/tmp/test.json


[1, {'1': 1, '2': 2}, 3]

In [6]:
# export
import yaml

def read_yaml(file):
    with open(file, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

def write_yaml(content, fpath):
    with open(fpath, 'w') as outfile:
        yaml.dump(content, outfile, default_flow_style=False)

In [7]:
# export
from torch_snippets.thinc_parser.parser import Config
Config = Config